In [1]:
import numpy as np

In [2]:
np.random.seed(0)

In [3]:
data = "hello world"
chars = list(set(data))
vocab_size = len(chars)

chars_to_indices = {char: i for i, char in enumerate(chars)}
indices_to_chars = {i: char for i, char in enumerate(chars)}

chars_to_indices, indices_to_chars

({'d': 0, ' ': 1, 'h': 2, 'l': 3, 'w': 4, 'o': 5, 'e': 6, 'r': 7},
 {0: 'd', 1: ' ', 2: 'h', 3: 'l', 4: 'w', 5: 'o', 6: 'e', 7: 'r'})

In [4]:
# RNN configs
embedding_size = vocab_size
hidden_size = 64
output_size = vocab_size

In [5]:
# Matrices
Wxh = np.random.randn(hidden_size, embedding_size)
Whh = np.random.randn(hidden_size, hidden_size)
bh = np.random.randn(1, hidden_size)

Why = np.random.randn(output_size, hidden_size)
by = np.random.randn(1, output_size)

In [6]:
def softmax(z):
    e_z = np.exp(z)
    return e_z / e_z.sum(axis=1)

def rnn(x_t, h_prev):
    assert x_t.shape == (1, embedding_size)
    assert h_prev.shape == (1, hidden_size)

    h_next = np.tanh(x_t @ Wxh.T + h_prev @ Whh.T + bh)
    probs = softmax(h_next @ Why.T + by)
    
    return probs, h_next

def sample(char, n):
    # sample rnn n times starting with the first char
    x = np.zeros((1, vocab_size))
    idx = chars_to_indices[char]
    x[:, idx] = 1 # one hot encoding
    h =  np.zeros((1, hidden_size)) # hidden state
    idxes = []
    
    for i in range(n):
        probs, h = rnn(x, h) # xt, hprev -> rnn -> probs, hnext
        idx = np.random.choice(vocab_size, p=probs.ravel()) # .ravel returns a 1d array
        
        x = np.zeros((1, vocab_size))
        x[:, idx] = 1
        idxes.append(idx)

    chars = "".join([indices_to_chars[i] for i in idxes])
    return chars

In [7]:
x_t = np.random.randn(1, embedding_size)
h_prev = np.random.randn(1, hidden_size)

probs, h_next = rnn(x_t, h_prev)
probs.shape, h_next.shape

((1, 8), (1, 64))

In [8]:
inp = [chars_to_indices[c] for c in data[:-1]]
target = [chars_to_indices[c] for c in data[1:]]

inp, target

([2, 6, 3, 3, 5, 1, 4, 5, 7, 3], [6, 3, 3, 5, 1, 4, 5, 7, 3, 0])

In [9]:
sample("h", 4)

'loeo'

In [12]:
lr = 1e-2
for e in range(100):
    # compute loss
    loss = 0
    xs, ps, hs = {}, {}, {}
    h = np.zeros((1, hidden_size))
    for t in range(len(inp)):
        idx = inp[t]
        x = np.zeros((1, vocab_size))
        x[:, idx] = 1
        probs, h = rnn(x, h)
        xs[t] = x
        ps[t] = probs # save probs (we'll use this in backprop)
        hs[t] = h

        # cross entropy
        pred = probs[0, target[t]]
        loss += -np.log(pred)

    # backprop, calculate gradients
    dL_dWxh = np.zeros_like(Wxh)
    dL_dWhh = np.zeros_like(Whh)
    dL_dbh = np.zeros_like(bh)
    dL_Why = np.zeros_like(Why)
    dL_dby = np.zeros_like(by)
    dF_dh = np.zeros((1, hidden_size))

    for t in reversed(range(len(inp))):
        dL_dz2 = np.copy(ps[t])
        target_idx = target[t]
        dL_dz2[:, target_idx] -= 1

        # print(dL_dz2.shape)

        a = hs[t] * dL_dz2.T
        # print(a.shape)

        # 2nd layer
        dL_Why += hs[t] * dL_dz2.T
        dL_dby += dL_dz2


        # 1st layer
        dh_dz1 = 1 - hs[t] ** 2
        dL_dh = dF_dh + (dL_dz2 @ Why) 

        # print((xs[t] * dh_dz1.T * dL_dh.T).shape)

        dL_dWxh += xs[t] * dh_dz1.T * dL_dh.T
        dL_dWhh +=  hs[t] * dh_dz1.T * dL_dh.T
        dL_dbh +=  dh_dz1 * dL_dh

        # raise RuntimeError

        dF_dh = dL_dh * (dh_dz1 @ Whh)

    # clip gradients
    for gradient in [dL_dWxh, dL_dWhh, dL_Why, dL_dbh, dL_dby]:
        np.clip(gradient, -5, 5, out=gradient)

    # gradient descent
    for weights, gradient in zip([Wxh, Whh, Why, bh, by], [dL_dWxh, dL_dWhh, dL_Why, dL_dbh, dL_dby]):
        weights -= lr * gradient

    # break
    
    print(f"[Loss]: {loss}")

[Loss]: 34.66832563164107
[Loss]: 30.01244846798594
[Loss]: 15.645373755979268
[Loss]: 19.971581194994467
[Loss]: 16.805803682483308
[Loss]: 9.408224506132868
[Loss]: 15.408018660165883
[Loss]: 13.132131775590006
[Loss]: 18.13153660975756
[Loss]: 16.19766730948435
[Loss]: 8.106177982276225
[Loss]: 5.733099683747487
[Loss]: 8.642087613127027
[Loss]: 5.566229676227321
[Loss]: 8.901615511658363
[Loss]: 4.796896136081292
[Loss]: 4.334701418111371
[Loss]: 3.1445381953574962
[Loss]: 2.7286873390105884
[Loss]: 2.506543504310333
[Loss]: 9.401562696718074
[Loss]: 12.020476091635857
[Loss]: 20.258330393832463
[Loss]: 13.578317095301559
[Loss]: 3.265648817178202
[Loss]: 2.5550103997640603
[Loss]: 15.303672286712096
[Loss]: 5.675782228356795
[Loss]: 21.860475767198363
[Loss]: 5.085170591650057
[Loss]: 21.601537383035524
[Loss]: 24.837529372343674
[Loss]: 11.707394669172452
[Loss]: 13.242564407857488
[Loss]: 5.380050193176152
[Loss]: 1.5306742029382931
[Loss]: 6.831515313061345
[Loss]: 7.5374346179

In [13]:
sample("h", 10)

'ello world'